In [ ]:
import pandas as pd
import os
import fastf1 as ff1
from fastf1.ergast import Ergast
import sys
from pathlib import Path

project_root = Path.cwd().parent  
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Added to path.")

# Now imports will work
from src.team_name_mapping import normalize_team_column

# Enable FastF1 cache
cache_path = "../data/fastf1_cache"
os.makedirs(cache_path, exist_ok=True)

ff1.Cache.enable_cache(cache_path)

✅ Added to path.


In [2]:
ergast = Ergast()

all_years = range(2014, 2025)
all_standings = []

for year in all_years:
    cs = ergast.get_constructor_standings(season=year)
    temp_df = cs.content[-1].copy()
    temp_df['season'] = year
    all_standings.append(temp_df)

standings_df = pd.concat(all_standings, ignore_index=True)

In [3]:
standings_df

,position,positionText,points,wins,constructorId,constructorUrl,constructorName,constructorNationality,season
0,1,1,701.0,16,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German,2014
1,2,2,405.0,3,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian,2014
2,3,3,320.0,0,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British,2014
3,4,4,216.0,0,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,2014
4,5,5,181.0,0,mclaren,http://en.wikipedia.org/wiki/McLaren,McLaren,British,2014
...,...,...,...,...,...,...,...,...,...
107,6,6,65.0,0,alpine,http://en.wikipedia.org/wiki/Alpine_F1_Team,Alpine F1 Team,French,2024
108,7,7,58.0,0,haas,http://en.wikipedia.org/wiki/Haas_F1_Team,Haas F1 Team,American,2024
109,8,8,46.0,0,rb,http://en.wikipedia.org/wiki/RB_Formula_One_Team,RB F1 Team,Italian,2024
110,9,9,17.0,0,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British,2024


In [4]:
all_years_data = []

for season in all_years:
    race_results = ergast.get_race_results(season=season)

    season_rows = []
    for rnd, df in enumerate(race_results.content, start=1):
        d = df.groupby(['constructorId', 'constructorName'], as_index=False)['points'].sum()
        d['round'] = rnd
        d['season'] = season
        season_rows.append(d)

    season_df = pd.concat(season_rows, ignore_index=True)

    # cumulative
    season_df['cumulative_points'] = (
        season_df
        .sort_values(['constructorId', 'round'])
        .groupby('constructorId')['points']
        .cumsum()
    )

    all_years_data.append(season_df)

full_df = pd.concat(all_years_data, ignore_index=True)

In [5]:
full_df

,constructorId,constructorName,points,round,season,cumulative_points
0,caterham,Caterham,0.0,1,2014,0.0
1,ferrari,Ferrari,18.0,1,2014,18.0
2,force_india,Force India,9.0,1,2014,9.0
3,lotus_f1,Lotus F1,0.0,1,2014,0.0
4,marussia,Marussia,0.0,1,2014,0.0
...,...,...,...,...,...,...
178,ferrari,Ferrari,22.0,2,2024,49.0
179,haas,Haas F1 Team,1.0,2,2024,1.0
180,mclaren,McLaren,16.0,2,2024,28.0
181,mercedes,Mercedes,10.0,2,2024,26.0


In [6]:
df=standings_df.copy()

In [7]:
df.head()

,position,positionText,points,wins,constructorId,constructorUrl,constructorName,constructorNationality,season
0,1,1,701.0,16,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German,2014
1,2,2,405.0,3,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian,2014
2,3,3,320.0,0,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British,2014
3,4,4,216.0,0,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,2014
4,5,5,181.0,0,mclaren,http://en.wikipedia.org/wiki/McLaren,McLaren,British,2014


In [9]:

 # Normalize team names
df = normalize_team_column(df, col='constructorName')

# Show me what we have
print("Years available:")
print(sorted(df['season'].unique()))

print("\nTeams after normalization:")
print(sorted(df['constructorName'].unique()))

print("\nFirst 20 rows:")
print(df.head(20))

print("\nData shape:")
print(df.shape)

print("\nColumns:")
print(df.columns.tolist())


Unknown team names encountered (using uppercase fallback): ['ALPINE', 'ASTON MARTIN', 'CATERHAM', 'FERRARI', 'HAAS', 'MANOR MARUSSIA', 'MCLAREN', 'MERCEDES', 'RED BULL', 'SAUBER', 'WILLIAMS']


Years available:
[np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

Teams after normalization:
['ALPINE', 'ASTON MARTIN', 'CATERHAM', 'FERRARI', 'HAAS', 'MANOR MARUSSIA', 'MCLAREN', 'MERCEDES', 'RB', 'RED BULL', 'SAUBER', 'WILLIAMS']

First 20 rows:
    position positionText  points  wins constructorId  \
0          1            1   701.0    16      mercedes   
1          2            2   405.0     3      red_bull   
2          3            3   320.0     0      williams   
3          4            4   216.0     0       ferrari   
4          5            5   181.0     0       mclaren   
5          6            6   155.0     0   force_india   
6          7            7    30.0     0    toro_rosso   
7          8            8    10.0     0      lotus_f1   
8          9            9     2.0     0      marussia   
9         10           10     0.0     0        sauber

In [10]:
# Create year-by-year summary
summary = (
    df.groupby(['season', 'constructorName'])
    .agg({
        'points': 'sum',
        'wins': 'sum',
        'position': 'first'
    })
    .reset_index()
    .sort_values(['season', 'position'])
)

# Focus on SCM-relevant period (2017-2024)
scm_teams = ['MCLAREN', 'SAUBER', 'RB', 'ALPINE', 'HAAS', 
             'ASTON MARTIN', 'WILLIAMS']

scm_data = summary[
    (summary['season'] >= 2017) & 
    (summary['constructorName'].isin(scm_teams))
]

print("🏎️ SCM-Relevant Teams (2017-2024) - Constructor Points:")
pivot = scm_data.pivot_table(
    index='constructorName', 
    columns='season', 
    values='points',
    fill_value=0
)
print(pivot)

print("\n\n🎯 McLaren Trajectory (Full History 2014-2024):")
mclaren = summary[summary['constructorName'] == 'MCLAREN'][
    ['season', 'position', 'points', 'wins']
]
print(mclaren.to_string(index=False))

print("\n\n📊 Potential Donor Pool (2017-2024):")
donors = ['SAUBER', 'RB', 'ALPINE', 'HAAS']
donor_data = scm_data[scm_data['constructorName'].isin(donors)]
donor_pivot = donor_data.pivot_table(
    index='constructorName',
    columns='season',
    values='points',
    fill_value=0
)
print(donor_pivot)

print("\n\n🔍 Check: Teams that existed in all years 2017-2024:")
team_years = scm_data.groupby('constructorName')['season'].nunique()
print(team_years[team_years == 8])  # Should have 8 years each

🏎️ SCM-Relevant Teams (2017-2024) - Constructor Points:
season            2017   2018   2019   2020   2021   2022   2023   2024
constructorName                                                        
ALPINE            57.0  122.0   91.0  181.0  155.0  173.0  120.0   65.0
ASTON MARTIN     187.0  111.0   73.0  195.0   77.0   55.0  280.0   94.0
HAAS              47.0   93.0   28.0    3.0    0.0   37.0   12.0   58.0
MCLAREN           30.0   62.0  145.0  202.0  275.0  159.0  302.0  666.0
RB                53.0   33.0   85.0  107.0  142.0   35.0   25.0   46.0
SAUBER             5.0   48.0   57.0    8.0   13.0   55.0   16.0    4.0
WILLIAMS          83.0    7.0    1.0    0.0   23.0    8.0   28.0   17.0


🎯 McLaren Trajectory (Full History 2014-2024):
 season  position  points  wins
   2014         5   181.0     0
   2015         9    27.0     0
   2016         6    76.0     0
   2017         9    30.0     0
   2018         7    62.0     0
   2019         4   145.0     0
   2020         3   202

In [11]:
# Show me the structure of your current DataFrame
print(df.info())
print("\nSample of raw data:")
print(df.head(10))

# Confirm we have what we need for 2017-2020 pre-treatment
pre_treatment = df[(df['season'] >= 2017) & (df['season'] <= 2020)]
print("\nPre-treatment data (2017-2020):")
print(pre_treatment.groupby(['season', 'constructorName'])['points'].sum().unstack())

<class 'fastf1.ergast.interface.ErgastResultFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   position                112 non-null    int64  
 1   positionText            112 non-null    object 
 2   points                  112 non-null    float64
 3   wins                    112 non-null    int64  
 4   constructorId           112 non-null    object 
 5   constructorUrl          112 non-null    object 
 6   constructorName         112 non-null    object 
 7   constructorNationality  112 non-null    object 
 8   season                  112 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 8.0+ KB
None

Sample of raw data:
   position positionText  points  wins constructorId  \
0         1            1   701.0    16      mercedes   
1         2            2   405.0     3      red_bull   
2         3            3   320.0     0     

In [13]:
full_df = normalize_team_column(full_df, col='constructorName')
full_df

,constructorId,constructorName,points,round,season,cumulative_points
0,caterham,CATERHAM,0.0,1,2014,0.0
1,ferrari,FERRARI,18.0,1,2014,18.0
2,force_india,ASTON MARTIN,9.0,1,2014,9.0
3,lotus_f1,ALPINE,0.0,1,2014,0.0
4,marussia,MANOR MARUSSIA,0.0,1,2014,0.0
...,...,...,...,...,...,...
178,ferrari,FERRARI,22.0,2,2024,49.0
179,haas,HAAS,1.0,2,2024,1.0
180,mclaren,MCLAREN,16.0,2,2024,28.0
181,mercedes,MERCEDES,10.0,2,2024,26.0
